In [ ]:
from bs4 import BeautifulSoup
import requests

# pageNo = 1
for page in range(1,21) :
  url = 'https://finance.naver.com/item/sise_day.naver?code=005930&page=' + str(pageNo)
  headers = {'User-Agent' : 'Mozilla/5.0'}

  response = requests.get(url, headers=headers)

  soup = BeautifulSoup (response.text, 'html.parser')

  parsing_list = soup.find_all("tr")
  # len(parsing_list)

  isCheckNone = None

  for i in range (1, len(parsing_list)) :
    if (parsing_list[i].span != isCheckNone) :
      print(parsing_list[i].find_all("td", align="center")[0].text,
            parsing_list[i].find_all("td", class_="num")[0].text,
            parsing_list[i].find_all("td", class_="num")[1].text.strip())




In [42]:
# pandas를 이용해서 일별 시세 테이블 데이터 가져오기
import pandas as pd
df = pd.DataFrame()

for page in range(1,21) :
  url = 'https://finance.naver.com/item/sise_day.naver?code=005930&page=' + str(page)
  headers = {'User-Agent' : 'Mozilla/5.0'}

  response = requests.get(url, headers=headers)

  # df = df.append(pd.read_html(response.text, header=0)[0], ignore_index=True)
  df = pd.concat([df, pd.read_html(response.text, header=0)[0]], ignore_index=True)

# 결측치 제거
# df.isna().sum()
df = df.dropna()
# df.shape # (200, 7)



In [43]:
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.03.08,72800.0,600.0,72800.0,73400.0,72700.0,11165213.0
2,2024.03.07,72200.0,700.0,73100.0,73300.0,72200.0,14516963.0
3,2024.03.06,72900.0,800.0,73200.0,73500.0,72700.0,21547905.0
4,2024.03.05,73700.0,1200.0,74600.0,74800.0,73700.0,19505125.0
5,2024.03.04,74900.0,1500.0,74300.0,75000.0,74000.0,23210474.0
...,...,...,...,...,...,...,...
294,2023.05.22,68500.0,100.0,68400.0,69000.0,68000.0,14470308.0
295,2023.05.19,68400.0,2200.0,67800.0,68400.0,67500.0,20349345.0
296,2023.05.18,66200.0,1200.0,66000.0,66200.0,65700.0,14431704.0
297,2023.05.17,65000.0,400.0,65900.0,65900.0,64800.0,10745504.0


In [44]:
# 한글 컬럼명 => 영어로 변환
df = df.rename(columns={'날짜' : 'date', '종가' : 'close', '전일비' : 'diff', '시가' : 'open', '고가' : 'high', '저가' : 'low', '거래량' : 'volume'})

# data를 정수형으로  변환
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)

#date 컬럼을 date 타입으로 변환
df['date'] = pd.to_datetime(df['date'])

df = df.sort_values(by=['date'], ascending=True)

df

,date,close,diff,open,high,low,volume
298,2023-05-16,65400,900,65800,65900,65300,12334657
297,2023-05-17,65000,400,65900,65900,64800,10745504
296,2023-05-18,66200,1200,66000,66200,65700,14431704
295,2023-05-19,68400,2200,67800,68400,67500,20349345
294,2023-05-22,68500,100,68400,69000,68000,14470308
...,...,...,...,...,...,...,...
5,2024-03-04,74900,1500,74300,75000,74000,23210474
4,2024-03-05,73700,1200,74600,74800,73700,19505125
3,2024-03-06,72900,800,73200,73500,72700,21547905
2,2024-03-07,72200,700,73100,73300,72200,14516963


In [46]:
from datetime import datetime
df['date2'] = pd.to_datetime(df['date'])
# df['date2'].dt.year
# df['date2'].dt.month
# df['date2'].dt.day
df['year'] = df['date2'].dt.year
df

,date,close,diff,open,high,low,volume,date2,year
298,2023-05-16,65400,900,65800,65900,65300,12334657,2023-05-16,2023
297,2023-05-17,65000,400,65900,65900,64800,10745504,2023-05-17,2023
296,2023-05-18,66200,1200,66000,66200,65700,14431704,2023-05-18,2023
295,2023-05-19,68400,2200,67800,68400,67500,20349345,2023-05-19,2023
294,2023-05-22,68500,100,68400,69000,68000,14470308,2023-05-22,2023
...,...,...,...,...,...,...,...,...,...
5,2024-03-04,74900,1500,74300,75000,74000,23210474,2024-03-04,2024
4,2024-03-05,73700,1200,74600,74800,73700,19505125,2024-03-05,2024
3,2024-03-06,72900,800,73200,73500,72700,21547905,2024-03-06,2024
2,2024-03-07,72200,700,73100,73300,72200,14516963,2024-03-07,2024


In [52]:
import plotly.graph_objects as go

fig = go.Figure(data = [go.Candlestick(x=df['date'], open=df['open'],high=df['high'],low=df['low'],close=df['close'])], layout=go.Layout(title=go.layout.Title(text='Samsung Elec')))

fig.show()

